## Final QDA fitting

In [ ]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
import pandas as pd
from lib.trees import get_tree, parse_edge_list
import hashlib
from functools import partial
import random
from collections import defaultdict
import heapq
from typing import Tuple
import networkx as nx
from sklearn.metrics import accuracy_score
import pandas as pd
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor
import json
import time
from typing import Dict, Iterable
import joblib
import logging

2025-05-30 02:36:04.173293: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748565364.354982    4713 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748565364.409319    4713 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748565364.728305    4713 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748565364.728341    4713 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1748565364.728344    4713 computation_placer.cc:177] computation placer alr

In [3]:
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO,
    format="(%(asctime)s) %(levelname)s # %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

### Step 1: Load the data

We will start by loading the provided raw datasets, converting the `language` column to a factor, parsing the edge lists, and converting them to a `networkx` graph.

In [11]:
training = pd.read_csv("../data/train.csv")
training["language"] = training["language"].astype("category")
training["edgelist"] = training["edgelist"].apply(parse_edge_list)
training["tree"] = training["edgelist"].apply(get_tree)

testing = pd.read_csv("../data/test.csv")
testing["language"] = testing["language"].astype("category")
testing["edgelist"] = testing["edgelist"].apply(parse_edge_list)
testing["tree"] = testing["edgelist"].apply(get_tree)

training.head()

,language,sentence,n,edgelist,root,tree
0,Japanese,2,23,"[(6, 4), (2, 6), (2, 23), (20, 2), (15, 20), (...",10,"(6, 4, 2, 23, 20, 15, 3, 5, 14, 8, 12, 9, 18, ..."
1,Japanese,5,18,"[(8, 9), (14, 8), (4, 14), (5, 4), (1, 2), (6,...",10,"(8, 9, 14, 4, 5, 1, 2, 6, 17, 12, 3, 7, 11, 16..."
2,Japanese,8,33,"[(2, 10), (2, 14), (4, 2), (16, 4), (6, 16), (...",3,"(2, 10, 14, 4, 16, 6, 12, 32, 26, 3, 29, 27, 2..."
3,Japanese,11,30,"[(30, 1), (14, 24), (21, 14), (3, 21), (7, 3),...",30,"(30, 1, 14, 24, 21, 3, 7, 12, 27, 16, 8, 5, 26..."
4,Japanese,12,19,"[(19, 13), (16, 19), (2, 16), (4, 10), (4, 15)...",11,"(19, 13, 16, 2, 4, 10, 15, 5, 14, 12, 3, 1, 8,..."


Next, we'll unwind the trees into a new dataframe with one row per node, with columns for all the features engineered in the rest of the notebooks.

> This step takes a lot of time to run, so below we also provide two lines to load the precomputed datasets.

In [12]:
from typing import List


def get_longest_paths(graph) -> List[List]:
    """
    Find all longest simple path in an undirected graph.
    """
    longest_paths = []
    longest_path_length = 0
    for source in graph.nodes():
        for target in graph.nodes():
            if source == target:
                continue
            # Find all simple paths between source and target
            paths = nx.all_simple_paths(graph, source=source, target=target)
            for path in paths:
                if len(path) > longest_path_length:
                    longest_path_length = len(path)
                    longest_paths.clear()
                    longest_paths.append(path)
                elif len(path) == longest_path_length:
                    longest_paths.append(path)
    return longest_paths


def unwind_tree(testing: bool, row: pd.Series) -> pd.DataFrame:
    """
    Unwind a tree into a list of nodes, with their centrality scores.
    """
    tree: nx.Graph = row["tree"]
    language = row["language"]
    if not testing:
        root_node = row["root"]

    # Centrality measures
    eccentricity = nx.eccentricity(tree)  # Maximum distance to any other node in the tree.
    closeness_centrality = nx.closeness_centrality(tree)  # Reciprocal avg distance to all other nodes in the tree.
    degree_centrality = nx.degree_centrality(tree)  # Fraction of nodes that a node is connected to.
    betweenness_centrality = nx.betweenness_centrality(tree)  # Fraction of shortest paths that pass through.
    harmonic_centrality = nx.harmonic_centrality(tree)  # Average distance to all other nodes in the tree.
    pagerank = nx.pagerank(tree)  # PageRank algorithm, which measures the importance of nodes.
    katz_centrality = nx.katz_centrality(tree)  # It's a generalization of eigenvector centrality.
    current_flow_closeness = nx.current_flow_closeness_centrality(tree)  # Closeness cent. based on resistance.
    current_flow_betweenness = nx.current_flow_betweenness_centrality(tree)  # Betweenness cent. based on resistance.
    load_centrality = nx.load_centrality(tree)  # Similar to betweenness centrality.
    percolation_centrality = nx.percolation_centrality(tree)  # Proportion of "percolation" paths through a node.
    second_order_centrality = nx.second_order_centrality(tree)  # Std of return times in perpetual random walk.
    laplacian_centrality = nx.laplacian_centrality(tree)  # Centrality based on the Laplacian matrix.

    # Tree properties
    tree_diameter = nx.diameter(tree)  # Length of the longest path in the tree.
    # Get the centroids of the tree
    centroids = nx.center(tree)
    # Get the leaves of the tree
    leaves = [node for node, degree in tree.degree() if degree == 1]
    # Get the longest path in the tree
    longest_path = get_longest_paths(tree)
    longest_path_nodes = set()
    for path in longest_path:
        longest_path_nodes.update(path)

    rows = []
    for node in tree:
        # Create a tree rooted at the current node
        tree_rooted_at_node: nx.DiGraph = nx.bfs_tree(tree, source=node)
        # We can do this because we know it will be a tree, so no cycles exist
        # and thus, bfs_edges will give us a valid tree structure.

        subtrees: list[nx.DiGraph] = []
        for child in tree.neighbors(node):
            subtrees.append(nx.bfs_tree(tree, source=child))

        subtree_leaves = []
        for subtree in subtrees:
            subtree_leaves.append([n for n in subtree if subtree.out_degree[n] == 0])

        subtree_depths = []
        for subtree in subtrees:
            subtree_depths.append(nx.dag_longest_path_length(subtree))

        rows.append(
            {
                "row_index": row.name,
                "node": node,
                "is_root": node == root_node if not testing else None,
                # Global properties
                "language": language,
                "tree_diameter": tree_diameter,
                "tree_size": tree.number_of_nodes(),
                "tree_edges": tree.number_of_edges(),
                "number_of_centroids": len(centroids),
                "average_degree": sum(dict(tree.degree()).values()) / tree.number_of_nodes(),
                "number_of_leaves": len(leaves),
                # Local Node Properties
                "degree": tree.degree[node],
                "is_leaf": tree.degree[node] == 1,  # Only a leaf can have degree 1
                "is_centroid": node in centroids,
                # Path properties
                "distance_to_closest_centroid": min(
                    nx.shortest_path_length(tree, source=c, target=node) for c in centroids
                ),
                "distance_to_farthest_centroid": max(
                    nx.shortest_path_length(tree, source=c, target=node) for c in centroids
                ),
                "distance_to_closest_leaf": min(nx.shortest_path_length(tree, source=l, target=node) for l in leaves),
                "distance_to_farthest_leaf": max(nx.shortest_path_length(tree, source=l, target=node) for l in leaves),
                "is_in_longest_path": node in longest_path_nodes,
                # Properties of tree rooted at this node
                "tree_depth_if_root": nx.dag_longest_path_length(tree_rooted_at_node),
                "min_subtree_size_if_root": min(subtree.number_of_nodes() for subtree in subtrees) if subtrees else 0,
                "max_subtree_size_if_root": max(subtree.number_of_nodes() for subtree in subtrees) if subtrees else 0,
                "average_subtree_size_if_root": (
                    sum(subtree.number_of_nodes() for subtree in subtrees) / len(subtrees) if subtrees else 0
                ),
                "min_subtree_leaf_count_if_root": (min(len(l) for l in subtree_leaves) if subtree_leaves else 0),
                "max_subtree_leaf_count_if_root": (max(len(l) for l in subtree_leaves) if subtree_leaves else 0),
                "average_subtree_leaf_count_if_root": (
                    sum(len(l) for l in subtree_leaves) / len(subtree_leaves) if subtree_leaves else 0
                ),
                "min_subtree_depth_if_root": (min(subtree_depths) if subtree_depths else 0),
                "max_subtree_depth_if_root": (max(subtree_depths) if subtree_depths else 0),
                "average_subtree_depth_if_root": (sum(subtree_depths) / len(subtree_depths) if subtree_depths else 0),
                "depth_difference_if_root": (max(subtree_depths) - min(subtree_depths)) if subtree_depths else 0,
                "number_of_subtrees_if_root": len(subtrees),
                # Centrality measures
                "eccentricity": eccentricity[node],
                "closeness_centrality": closeness_centrality[node],
                "closeness_centrality_inverse": 1 / closeness_centrality[node] if closeness_centrality[node] > 0 else 0,
                "degree_centrality": degree_centrality[node],
                "harmonic_centrality": harmonic_centrality[node],
                "betweenness_centrality": betweenness_centrality[node],
                "pagerank": pagerank[node],
                "katz_centrality": katz_centrality[node],
                "current_flow_closeness": current_flow_closeness[node],
                "current_flow_betweenness": current_flow_betweenness[node],
                "load_centrality": load_centrality[node],
                "percolation_centrality": percolation_centrality[node],
                "second_order_centrality": second_order_centrality[node],
                "laplacian_centrality": laplacian_centrality[node],
            }
        )
    return pd.DataFrame(rows)

In [13]:
training = pd.concat(training.apply(partial(unwind_tree, False), axis=1).tolist(), ignore_index=True)
training["language"] = training["language"].astype("category")
training.to_csv("../data/cache/total_training_unwound.csv", index=False)

testing = pd.concat(testing.apply(partial(unwind_tree, True), axis=1).tolist(), ignore_index=True)
testing["language"] = testing["language"].astype("category")
testing.to_csv("../data/cache/total_testing_unwound.csv", index=False)

training.head()

,row_index,node,is_root,language,tree_diameter,tree_size,tree_edges,number_of_centroids,average_degree,number_of_leaves,...,harmonic_centrality,betweenness_centrality,pagerank,katz_centrality,current_flow_closeness,current_flow_betweenness,load_centrality,percolation_centrality,second_order_centrality,laplacian_centrality
0,0,6,False,Japanese,14,23,22,1,1.913043,6,...,5.823846,0.090909,0.048565,0.209086,0.007246,0.090909,0.090909,0.090909,98.762341,0.101449
1,0,4,False,Japanese,14,23,22,1,1.913043,6,...,4.561122,0.000000,0.027162,0.188298,0.006289,0.000000,0.000000,0.000000,112.481110,0.043478
2,0,2,False,Japanese,14,23,22,1,1.913043,6,...,6.991703,0.255411,0.066901,0.228660,0.008403,0.255411,0.255411,0.255411,84.451169,0.159420
3,0,23,False,Japanese,14,23,22,1,1.913043,6,...,5.157179,0.000000,0.025477,0.190256,0.007143,0.000000,0.000000,0.000000,100.149888,0.057971
4,0,20,False,Japanese,14,23,22,1,1.913043,6,...,7.146825,0.311688,0.042552,0.213357,0.009615,0.311688,0.311688,0.311688,71.147734,0.130435


This step takes a while, so here's a quick way to load the precomputed datasets:

In [4]:
# To load the data back from the CSV files:
training = pd.read_csv("../data/cache/total_training_unwound.csv")
training["language"] = training["language"].astype("category")
testing = pd.read_csv("../data/cache/total_testing_unwound.csv")
testing["language"] = testing["language"].astype("category")

We will use cross-validation to select the best feature set for the QDA model. To do this, we will use a greedy search over the feature combinations.

The following two functions will fit the QDA model over the five folds of the cross-validation in parallel, given a dataset.

In [45]:
def run_fold(
    fold: int, fold_path: Path, train_fold_data: pd.DataFrame, validation_fold_data: pd.DataFrame
) -> Dict[str, float]:

    # One-hot encode the training and validation data
    X_train = train_fold_data.drop(columns=["row_index", "node", "is_root"])
    if "language" in X_train.columns:
        X_train = pd.get_dummies(X_train, columns=["language"], drop_first=False)
    y_train = train_fold_data["is_root"]

    model = QuadraticDiscriminantAnalysis()

    model.fit(X_train, y_train)

    sentence_predictions = defaultdict(dict)
    sentence_real_root = {}
    X_val = validation_fold_data.drop(columns=["row_index", "node", "is_root"])
    if "language" in X_val.columns:
        X_val = pd.get_dummies(X_val, columns=["language"], drop_first=False)
    probabilities = model.predict_proba(X_val)
    for (_, row), probs in zip(validation_fold_data.iterrows(), probabilities):
        sentence_predictions[row["row_index"]][int(row["node"])] = probs[1]
        if row["is_root"]:
            sentence_real_root[row["row_index"]] = row["node"]

    if not set(sentence_predictions.keys()) == set(sentence_real_root.keys()):
        raise ValueError("Mismatch between sentence predictions and real roots.")

    def get_predicted_root(row: pd.Series) -> str:
        """
        Get the predicted root node for a sentence.
        """
        sentence_id = row.name
        probs = sentence_predictions[sentence_id]
        return max(probs.keys(), key=probs.get)

    validation_prediction = pd.DataFrame.from_dict(sentence_real_root, orient="index", columns=["root"])
    validation_prediction["predicted_root"] = validation_prediction.apply(get_predicted_root, axis=1)
    sentence_accuracy = accuracy_score(validation_prediction["root"], validation_prediction["predicted_root"])

    with open(fold_path / "metrics.json", "w") as f:
        json.dump({"accuracy": sentence_accuracy}, f, indent=4)

    del model

    return sentence_accuracy


def run_all_folds(
    cross_validation_folds: int, dataset: pd.DataFrame, row_indices: pd.Series, config_path: Path
) -> None:
    accuracies = []
    # Separate cross-validation data
    with ProcessPoolExecutor(max_workers=cross_validation_folds) as executor:
        jobs = []
        for fold in range(cross_validation_folds):
            fold_path = config_path / f"fold-{fold}"
            fold_path.mkdir(parents=True, exist_ok=True)

            fold_row_indices = row_indices[fold::cross_validation_folds]
            with open(fold_path / "fold_validation_row_indices.json", "w") as f:
                json.dump(fold_row_indices.tolist(), f)

            train_fold_data = dataset[~dataset["row_index"].isin(fold_row_indices)]
            validation_fold_data = dataset[dataset["row_index"].isin(fold_row_indices)]
            job = executor.submit(run_fold, fold, fold_path, train_fold_data, validation_fold_data)
            jobs.append(job)
        for fold, job in enumerate(jobs):
            accuracy = job.result()
            accuracies.append(accuracy)
            logger.info(f"Fold {fold + 1}/{cross_validation_folds} - Accuracy: {accuracy:.4f}")

    with open(config_path / "metrics.json", "w") as f:
        json.dump({"accuracies": accuracies, "average_accuracy": sum(accuracies) / len(accuracies)}, f, indent=4)

    return sum(accuracies) / len(accuracies)

And the following objects are helpers to run and store the current state of the greedy search algorithm.

In [50]:
class FeatureSet:
    def __init__(self, features: Iterable[str]):
        self.features = set(features)
        self.hash = self.stable_hash(self.features)

    @staticmethod
    def stable_hash(fs: frozenset) -> int:
        # Convert to a sorted, reproducible representation
        sorted_items = sorted(repr(item) for item in fs)
        serialized = "|".join(sorted_items)  # Any stable separator
        return int(hashlib.sha256(serialized.encode("utf-8")).hexdigest(), 16)

    def __hash__(self):
        return self.hash

    def __eq__(self, other):
        return isinstance(other, FeatureSet) and self.features == other.features

    def __lt__(self, other):
        # Absurd method, but ensures that FeatureSet can be used in a heap
        return self.hash < other.hash


class FeatureTree:
    def __init__(self, available_features: List[str]):
        self.available_features = available_features
        self._explored_feature_sets: Dict[FeatureSet, float] = {}
        self._explored_feature_set_heap: List[Tuple[float, FeatureSet]] = []
        self._explored_feature_set_heap_size = 0
        self.random = random.Random()

    def add_feature_set(self, feature_set: FeatureSet, score: float) -> None:
        self._explored_feature_sets[feature_set] = score
        heapq.heappush(self._explored_feature_set_heap, (-score, feature_set))  # We use negative score for a max-heap
        self._explored_feature_set_heap_size += 1

    def is_empty(self) -> bool:
        return self._explored_feature_set_heap_size == 0

    def get_best_feature_set(self) -> FeatureSet:
        if not self._explored_feature_set_heap:
            raise ValueError("No explored feature sets available.")
        # Get the best feature set (highest score)
        best_score, best_feature_set = self._explored_feature_set_heap[0]
        return best_feature_set

    def remove_feature_set_from_heap(self, feature_set: FeatureSet) -> float:
        if feature_set not in self._explored_feature_sets:
            raise ValueError("Feature set not found.")
        # Remove the feature set from the heap
        self._explored_feature_sets.pop(feature_set)
        self._explored_feature_set_heap = [
            (score, fs) for score, fs in self._explored_feature_set_heap if fs != feature_set
        ]
        heapq.heapify(self._explored_feature_set_heap)
        self._explored_feature_set_heap_size -= 1
        return feature_set

    def branch_feature_set(self, feature_set: FeatureSet) -> FeatureSet:
        """
        Create a new feature set by adding or subtracting one feature from the available features to the current feature set.
        """
        random_feature_order = self.random.sample(self.available_features, len(self.available_features))

        for feature in random_feature_order:
            if feature not in feature_set.features:
                new_features = feature_set.features | {feature}
                new_features = FeatureSet(new_features)
                if new_features not in self._explored_feature_sets:
                    return new_features
            else:
                new_features = feature_set.features - {feature}
                new_features = FeatureSet(new_features)
                if new_features not in self._explored_feature_sets:
                    return new_features
        return None  # No more features to branch from this set

Now it's time for the magic. We'll run the greedy search algorithm indefinitely until it runs out of different combinations, 
which, based on our calculations, would take around 400000 years.

Each step in the process will be cached to disk, so if it stops for any reason, we can resume it later.

In [14]:
model_path = Path("../data/models/qda/featureselection")
cross_validation_folds = 5

row_indices = training["row_index"].unique()
# We'll separate based on row indices, because that's what we have now. Ideally
# we would separate based on sentence id, but we don't have that in the data now

all_features = training.columns.difference(["row_index", "node", "is_root"]).tolist()

In [ ]:
feature_tree = FeatureTree(all_features)
feature_tree.random.seed(42)  # For reproducibility

# Compute the first point in the feature tree to start the search
config_path = model_path / f"configuration-base"
if not config_path.exists():
    config_path.mkdir(parents=True, exist_ok=True)
    with open(config_path / "features.json", "w") as f:
        json.dump(all_features, f, indent=4)

    average_accuracy = run_all_folds(cross_validation_folds, training, row_indices, config_path)
else:
    with open(config_path / "metrics.json", "r") as f:
        average_accuracy = json.load(f)["average_accuracy"]
logger.info(f"Base configuration average accuracy: {average_accuracy:.4f}")

feature_tree.add_feature_set(FeatureSet(all_features), average_accuracy)

j = 1
while True:
    if feature_tree.is_empty():
        logger.info("No more feature sets to explore. Stopping.")
        break
    best_feature_set = feature_tree.get_best_feature_set()
    # Branch the feature set to get a new one
    new_feature_set = feature_tree.branch_feature_set(best_feature_set)
    # If we found no new feature set, this means that all the branches from
    # the current best feature set have been explored.
    if new_feature_set is None:
        logger.info("Current best feature set has no more branches to explore.")
        feature_tree.remove_feature_set_from_heap(best_feature_set)
        continue

    logger.info(f"Feature Set {j}: {new_feature_set.features}")
    config_path = model_path / f"configuration-{new_feature_set.hash}"
    conf_start = time.time()
    if config_path.exists():
        logger.info(f"Configuration {new_feature_set.hash} already exists. Skipping.")
        with open(config_path / "metrics.json", "r") as f:
            average_accuracy = json.load(f)["average_accuracy"]
    else:
        config_path.mkdir(parents=True, exist_ok=True)
        with open(config_path / "features.json", "w") as f:
            json.dump(list(new_feature_set.features), f, indent=4)

        train_data = training[
            list(training.columns.intersection(new_feature_set.features)) + ["row_index", "node", "is_root"]
        ]

        average_accuracy = run_all_folds(cross_validation_folds, train_data, row_indices, config_path)
    feature_tree.add_feature_set(new_feature_set, average_accuracy)
    logger.info(
        f"Feature Set {j} completed in {time.time() - conf_start:.2f} seconds. Average Accuracy: {average_accuracy:.4f}"
    )
    j += 1

(2025-05-30 02:02:19) INFO # Base configuration average accuracy: 0.3211
(2025-05-30 02:02:19) INFO # Feature Set 1: {'load_centrality', 'number_of_leaves', 'degree_centrality', 'depth_difference_if_root', 'betweenness_centrality', 'second_order_centrality', 'distance_to_closest_centroid', 'language', 'tree_diameter', 'current_flow_betweenness', 'average_subtree_leaf_count_if_root', 'is_in_longest_path', 'max_subtree_size_if_root', 'closeness_centrality', 'degree', 'closeness_centrality_inverse', 'distance_to_farthest_centroid', 'tree_depth_if_root', 'min_subtree_depth_if_root', 'percolation_centrality', 'katz_centrality', 'max_subtree_leaf_count_if_root', 'tree_edges', 'harmonic_centrality', 'current_flow_closeness', 'number_of_centroids', 'max_subtree_depth_if_root', 'laplacian_centrality', 'min_subtree_size_if_root', 'distance_to_closest_leaf', 'eccentricity', 'min_subtree_leaf_count_if_root', 'number_of_subtrees_if_root', 'is_leaf', 'average_subtree_size_if_root', 'average_subtree_

Finally, let's select the best combination of features from the created trees:

In [11]:
accuracy_scores = []
for config_folder in model_path.glob("configuration-*"):
    with open(config_folder / "metrics.json", "r") as f:
        metrics = json.load(f)
        accuracy_scores.append((config_folder.name, metrics["average_accuracy"]))
best_file, best_accuracy = max(accuracy_scores, key=lambda x: x[1])
logger.info(f"Best features: {best_file} with accuracy {best_accuracy:.4f}")

(2025-05-30 02:38:54) INFO # Best features: configuration-58771610805917274678274061772213642871591361859644864657621009439125096213338 with accuracy 0.3374


In [16]:
# Now load the best features
best_features_path = model_path / best_file / "features.json"
with open(best_features_path, "r") as f:
    best_features = json.load(f)
logger.info(f"Best features loaded: {best_features}")
# Features that were removed from the original set
removed_features = set(all_features) - set(best_features)
logger.info(f"Removed features: {removed_features}")

(2025-05-30 02:39:55) INFO # Best features loaded: ['load_centrality', 'number_of_leaves', 'degree_centrality', 'depth_difference_if_root', 'betweenness_centrality', 'second_order_centrality', 'distance_to_closest_centroid', 'language', 'tree_diameter', 'current_flow_betweenness', 'average_subtree_leaf_count_if_root', 'is_in_longest_path', 'max_subtree_size_if_root', 'tree_size', 'closeness_centrality', 'closeness_centrality_inverse', 'distance_to_farthest_centroid', 'tree_depth_if_root', 'min_subtree_depth_if_root', 'percolation_centrality', 'katz_centrality', 'max_subtree_leaf_count_if_root', 'tree_edges', 'harmonic_centrality', 'number_of_centroids', 'max_subtree_depth_if_root', 'min_subtree_size_if_root', 'distance_to_closest_leaf', 'eccentricity', 'min_subtree_leaf_count_if_root', 'number_of_subtrees_if_root', 'is_leaf', 'average_subtree_size_if_root', 'average_subtree_depth_if_root', 'pagerank', 'is_centroid', 'distance_to_farthest_leaf', 'average_degree']
(2025-05-30 02:39:55) I

And finally, let's train the QDA model on the best feature set and evaluate it on the test set.

In [19]:
# Remove the removed features from the training and testing data
new_training = training.drop(columns=removed_features, errors="ignore")
new_testing = testing.drop(columns=removed_features, errors="ignore")

# Then train the QDA model on the new training data
X_train = new_training.drop(columns=["row_index", "node", "is_root"])
if "language" in X_train.columns:
    X_train = pd.get_dummies(X_train, columns=["language"], drop_first=False)
y_train = new_training["is_root"]
model = QuadraticDiscriminantAnalysis()
model.fit(X_train, y_train)
# Save the model
joblib.dump(model, model_path.parent / "qda_model.joblib")

/home/mparcerisa/Documents/Work/MDS/S2/ML/SyntacticTreeRooting/.venv/lib/python3.12/site-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 0 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(
/home/mparcerisa/Documents/Work/MDS/S2/ML/SyntacticTreeRooting/.venv/lib/python3.12/site-packages/sklearn/discriminant_analysis.py:1024: LinAlgWarning: The covariance matrix of class 1 is not full rank. Increasing the value of parameter `reg_param` might help reducing the collinearity.
  warnings.warn(


['../data/models/qda/qda_model.joblib']

In [27]:
# Finally, use the model to predict on the testing data
X_test = new_testing.drop(columns=["row_index", "node", "is_root"])
if "language" in X_test.columns:
    X_test = pd.get_dummies(X_test, columns=["language"], drop_first=False)
predictions = model.predict(X_test)

# Now predict the roots for the testing data
sentence_predictions = defaultdict(dict)
probabilities = model.predict_proba(X_test)
for (_, row), probs in zip(new_testing.iterrows(), probabilities):
    sentence_predictions[row["row_index"]][int(row["node"])] = probs[1]

sentence_roots = {}
for i, probs in sentence_predictions.items():
    sentence_roots[i] = max(probs.keys(), key=probs.get)

validation_prediction = pd.DataFrame.from_dict(sentence_roots, orient="index", columns=["root"])
validation_prediction

,root
0,37
1,46
2,2
3,15
4,1
...,...
10390,12
10391,7
10392,26
10393,19


In [ ]:
# Finally, we can join the predictions with the original testing data to match the id and the root

original_testing = pd.read_csv("../data/test.csv")
original_testing["row_index"] = original_testing.index
validation_prediction = validation_prediction.join(original_testing[["row_index", "id"]].set_index("row_index"))
validation_prediction = validation_prediction.reset_index(drop=True)
validation_prediction = validation_prediction[["id", "root"]]
validation_prediction.to_csv("../data/submissions/submission-qda.csv", index=False)